In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
ticker = yf.Ticker('EVO.ST')
df = ticker.history('5y')

In [7]:
X = df.drop('Close', axis=1)
y = df['Close']

In [8]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)  # Reshape y to a column vector
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)  # Reshape y to a column vector



In [9]:
# Create DataLoader for batch processing during training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [10]:
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.fc(x)

In [11]:
input_size = X_train_tensor.shape[1]
model = RegressionModel(input_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
# Training the model
num_epochs = 100

for epoch in range(num_epochs):
    model.train()  # Set the model in training mode
    total_loss = 0

    for inputs, targets in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Calculate the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the weights
        total_loss += loss.item()

    # Print the average loss for the epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")


Epoch 1/100, Loss: 838161.6796875
Epoch 2/100, Loss: 841038.265625
Epoch 3/100, Loss: 839556.703125
Epoch 4/100, Loss: 837595.0234375
Epoch 5/100, Loss: 842336.35546875
Epoch 6/100, Loss: 840217.83984375
Epoch 7/100, Loss: 842862.26953125
Epoch 8/100, Loss: 840785.81640625
Epoch 9/100, Loss: 841000.52734375
Epoch 10/100, Loss: 839449.84375
Epoch 11/100, Loss: 837816.484375
Epoch 12/100, Loss: 841037.5625
Epoch 13/100, Loss: 843080.1171875
Epoch 14/100, Loss: 841035.0234375
Epoch 15/100, Loss: 839005.98046875
Epoch 16/100, Loss: 839263.26953125
Epoch 17/100, Loss: 835422.36328125
Epoch 18/100, Loss: 838009.52734375
Epoch 19/100, Loss: 842749.27734375
Epoch 20/100, Loss: 840232.890625
Epoch 21/100, Loss: 838888.87890625
Epoch 22/100, Loss: 839244.75
Epoch 23/100, Loss: 838940.41796875
Epoch 24/100, Loss: 838422.79296875
Epoch 25/100, Loss: 843082.859375
Epoch 26/100, Loss: 838019.51953125
Epoch 27/100, Loss: 837470.109375
Epoch 28/100, Loss: 842843.28125
Epoch 29/100, Loss: 837677.882812

In [13]:
# Evaluating the model on the test data
model.eval()  # Set the model in evaluation mode
with torch.no_grad():
    y_pred = model(X_test_tensor)
    mse = mean_squared_error(y_test_tensor, y_pred)
    r2 = r2_score(y_test_tensor, y_pred)

    print(f"Mean Squared Error (MSE): {mse}")
    print(f"R-squared (R2): {r2}")

# Now, you can use the trained model for predicting future stock prices
# Create a new DataFrame called 'future_data' containing the features (OHLC, Volume, Dividend, Stock_Splits) for the future dates.

# Assuming 'future_data' is the DataFrame containing future data
future_data_scaled = scaler.transform(future_data)
future_data_tensor = torch.tensor(future_data_scaled, dtype=torch.float32)
future_close_prices = model(future_data_tensor).numpy().flatten()

# 'future_close_prices' will contain the predicted Close prices for the future dates.

Mean Squared Error (MSE): 720393.8125
R-squared (R2): -2.2707482429253556


NameError: name 'future_data' is not defined